In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import itertools


SyntaxError: invalid syntax (1805036053.py, line 8)

In [ ]:
file = '../datasets/Cirrhosis.csv'
data = pd.read_csv(file)

data.dropna(inplace=True)
data.drop(columns=['ID'], inplace=True)
data['Age'] = data['Age'] / 365.25  # Convert age from days to years

categorical_cols = ['Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']
numerical_cols = data.columns.difference(categorical_cols + ['Stage']).tolist()

# One-hot encoding
one_hot_encoded = [pd.get_dummies(data[col], prefix=col) for col in categorical_cols]
X_categorical = pd.concat(one_hot_encoded, axis=1)

# Scaling
scaler = StandardScaler()
X_numerical = pd.DataFrame(scaler.fit_transform(data[numerical_cols]), columns=numerical_cols)

# Combine processed categorical and numerical data
X_processed = pd.concat([X_numerical, X_categorical], axis=1)

In [ ]:
X = X_processed
y = data['Stage']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
models = {
    'NeuralNetwork': NeuralNetwork(input_size=X_train.shape[1], hidden_size=50, output_size=1),
    'DecisionTree': DecisionTree(max_depth=5),
    'LogisticRegression': LogisticRegression(learning_rate=0.01, epochs=1000)
}

In [ ]:
def manual_grid_search(model, param_grid, X_train, y_train, X_test, y_test):
    best_score = -np.inf
    best_params = None
    best_model = None

    # Iterate over each parameter combination
    keys, values = zip(*param_grid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        print(f"Testing parameters: {params}")
        model.set_params(**params)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        score = accuracy_score(y_test, predictions)

        print(f"Accuracy: {score}")

        if score > best_score:
            best_score = score
            best_params = params
            best_model = model

    print(f"Best parameters: {best_params}")
    print(f"Best accuracy: {best_score}")
    return best_model

In [ ]:
best_estimators = {}
for model_name, model in models.items():
    print(f"\nStarting grid search for {model_name}")
    best_model = manual_grid_search(model, param_grids[model_name], X_train, y_train, X_test, y_test)
    best_estimators[model_name] = best_model
    y_pred = best_model.predict(X_test)
    print(f"Classification report for {model_name}:\n", classification_report(y_test, y_pred))
    print(f"Confusion matrix for {model_name}:\n", confusion_matrix(y_test, y_pred))
    print("="*60)